In [2]:
import os
import tensorflow as tf
from tensorflow import keras

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# Warum auch immer, bekomme ich einen fehler wenn ich das hier nicht drinne habe. 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Speichern des Modells über callback während und am ende des Trainings

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images,test_labels), callbacks=[cp_callback]) 

Epoch 1/10
 1/32 [..............................] - ETA: 0s - loss: 2.2203 - sparse_categorical_accuracy: 0.2500
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 10ms/step - loss: 1.1253 - sparse_categorical_accuracy: 0.6830 - val_loss: 0.6806 - val_sparse_categorical_accuracy: 0.7920
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4103 - sparse_categorical_accuracy: 0.8705
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.4213 - sparse_categorical_accuracy: 0.8670 - val_loss: 0.5131 - val_sparse_categorical_accuracy: 0.8350
Epoch 3/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2893 - sparse_categorical_accuracy: 0.9271
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 0.2881 - sparse_categorical_accuracy: 0.9260 - val_loss: 0.4759 - val_sparse_categorical_accuracy: 0.8470
Epoch 4/10
 1/3

In [6]:
# Prüfen ob das Laden funktioniert, erst neues Modell erstellen und das einfach mal testen...

model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Neues Modell, accuracy: {:5.2f}%".format(100*acc)) # überraschung

32/32 - 0s - loss: 2.3708 - sparse_categorical_accuracy: 0.0680
Neues Modell, accuracy:  6.80%


In [7]:
# Jetzt das Laden der weights von eben
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4100 - sparse_categorical_accuracy: 0.8650
Restored model, accuracy: 86.50%


In [8]:
# Speichern der Checkpoints alle 5 Epochen und unter eigenen Namen
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True, period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels, epochs=50, callbacks=[cp_callback], validation_data=(test_images,test_labels), verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [10]:
# Testen obs läuft...
latest = tf.train.latest_checkpoint(checkpoint_dir)

model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4948 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


In [11]:
# Manuelles speichern..

model.save_weights('./checkpoints/my_checkpoint')
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4948 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%


In [12]:
# Es kann auch das ganze Modell gespeichert werden, wenn man nicht den Python code zur verfügung stellen möchte
# Dabei sollte man auf custom made layers achten, da diese nicht so einfach gespeichert werden können

model = create_model()
model.fit(train_images, train_labels, epochs=5)

model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 1ms/step - loss: 1.1664 - sparse_categorical_accuracy: 0.6680
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4362 - sparse_categorical_accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2875 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2176 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1560 - sparse_categorical_accuracy: 0.9690
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [13]:
# Laden des kompletten Modells:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4103 - sparse_categorical_accuracy: 0.8730
Restored model, accuracy: 87.30%
(1000, 10)


In [15]:
# Keras bietet noch die Methode: model.save(NAME.h5)
# Dadurch wird ALLES gespeichert und kann somit wiederhergestellt werden